In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

## 1. Crawl single site

In [2]:
def get_title(url): 
    res = requests.get(url) #get the website, return request.Response object
    #print(res.status_code) #statu_code: return 200(found web), 404(not found)
    soup = BeautifulSoup(res.text, 'html.parser')
    us_news_div = soup.find_all('div', re.compile('article-content'))
    
    title_list = []
    for i in range(len(us_news_div)):
        us_news_h3 = us_news_div[i].find_all('h3', recursive=False)
        us_news_a = us_news_h3[0].find_all('a', recursive=False)
        for index, item in enumerate(us_news_a[:]):
            title = item.text.strip()
            title_list.append(title)
    return title_list

In [3]:
def get_url(title_list):
    title_name = ''
    url_list = []
    pattern = re.compile('([^\s\w]|_)+')

    for i in range(len(title_list)):
        title = re.sub("[‘’]", '', title_list[i])
        #print(title)
        strippedList = pattern.sub(' ', title)
        a = strippedList.split(" ")
        empty_string = ''
        if empty_string in a:
            a = [x for x in a if x != '']
            title_name = '-'.join(a)
        else:
            title_name = '-'.join(a)
        url = 'https://www.infowars.com/'+title_name
        url_list.append(url)        
    return url_list

In [4]:
def get_content(url):
    res_content = requests.get(url) #get the website, return request.Response object
    if(res_content.status_code == 200):
        soup2 = BeautifulSoup(res_content.text, 'html.parser')
        article = soup2.find_all('article')
        content = article[0].find_all('p')
        full_content = ""
        for index, item in enumerate(content[:]):
            content2 = item.text.strip()   
            full_content = full_content+content2
        return full_content
    else:
        return

In [5]:
url1 = 'https://www.infowars.com/news/'

In [6]:
content_list = []
title_list = get_title(url1)
url_list = get_url(title_list)
for i in range(len(title_list)):
    content_list.append(get_content(url_list[i]))

In [7]:
#Dataframe: News | URL | Content
data = {'Title':title_list,'URL':url_list,'Content':content_list}
df = pd.DataFrame(data)
df = df[['Title', 'URL', 'Content']]
df.head()

,Title,URL,Content
0,Parkland Father Lauds George H.W. Bush’s Resig...,https://www.infowars.com/Parkland-Father-Lauds...,"A father who lost a child during the Parkland,..."
1,FBI Raids Home Of Whistleblower On Clinton Fou...,https://www.infowars.com/FBI-Raids-Home-Of-Whi...,FBI agents raided the home of a recognized Dep...
2,Ted Cruz Predicts ‘Anger And Rage’ May Lead To...,https://www.infowars.com/Ted-Cruz-Predicts-Ang...,Republican Sen. Ted Cruz told students Thursda...
3,Former President George H.W. Bush dies at age 94,https://www.infowars.com/Former-President-Geor...,"George Herbert Walker Bush, whose lone term as..."
4,Google Debated Gaming Search Results to Bury C...,https://www.infowars.com/Google-Debated-Gaming...,"Following Donald Trump’s election win, Google ..."


In [8]:
df.to_csv('result.csv')

# 2. Crawl Multiple Web Pages
## Dataframe: News | URL | Content

In [9]:
url = 'https://www.infowars.com'
res = requests.get(url) #get the website, return request.Response object
print(res.status_code) #statu_code: return 200(found web), 404(not found)

200


In [10]:
#print(res.text)

In [11]:
def get_tag(res, tag):   
    soup = BeautifulSoup(res.text, 'html.parser')
    tag_li = soup.find_all('li', re.compile(tag))
    pattern = re.compile('([^\s\w]|_)+')
    tag_list = []
    
    for i in range(len(tag_li)):
        tag_a = tag_li[i].find_all('a', recursive=False)
        for index, item in enumerate(tag_a[:]):
            tag = item.text.strip()
            tag = pattern.sub(' ', re.sub("[‘’.]", '', tag))
            tag_list.append(tag)

    return list(set(tag_list))

In [12]:
def get_category_url(title_list):
    title_name = ''
    url_list = []
    pattern = re.compile('([^\s\w]|_)+')

    for i in range(len(title_list)):
        title = re.sub("[‘’]", '', title_list[i])
        #print(title)
        strippedList = pattern.sub(' ', title)
        a = strippedList.split(" ")
        empty_string = ''
        if empty_string in a:
            a = [x for x in a if x != '']
            title_name = '-'.join(a)
        else:
            title_name = '-'.join(a)
        url = 'https://www.infowars.com/category/'+title_name
        url_list.append(url)        
    return url_list

In [13]:
news_tag = 'menu-item menu-item-type-taxonomy menu-item-object-category'
category_url_list = get_category_url(get_tag(res, news_tag))
category_url_list

['https://www.infowars.com/category/Science-Technology',
 'https://www.infowars.com/category/World-News',
 'https://www.infowars.com/category/Economy',
 'https://www.infowars.com/category/Health',
 'https://www.infowars.com/category/Hot-News',
 'https://www.infowars.com/category/World-at-War',
 'https://www.infowars.com/category/Globalism',
 'https://www.infowars.com/category/Special-Reports',
 'https://www.infowars.com/category/Government',
 'https://www.infowars.com/category/US-News']

In [14]:
#make sure the news webpages exist
for i in range(len(category_url_list)):
    res = requests.get(category_url_list[i]) #get the website, return request.Response object
    print(res.status_code) #statu_code: return 200(found web), 404(not found)

200
200
200
200
200
200
200
200
200
200


In [15]:
category_url_list

['https://www.infowars.com/category/Science-Technology',
 'https://www.infowars.com/category/World-News',
 'https://www.infowars.com/category/Economy',
 'https://www.infowars.com/category/Health',
 'https://www.infowars.com/category/Hot-News',
 'https://www.infowars.com/category/World-at-War',
 'https://www.infowars.com/category/Globalism',
 'https://www.infowars.com/category/Special-Reports',
 'https://www.infowars.com/category/Government',
 'https://www.infowars.com/category/US-News']

In [16]:
get_title(category_url_list[0])

['“It Was Anarchy” As Mag 7 Quake Rocks Anchorage; Trump Promises Federal Aid',
 'NASA announces the nine companies competing to put us back on the Moon',
 'How many photons has the universe produced in its life?',
 'Discovery Jeopardizes East Africa’s “cradle of mankind” Title',
 'NASA chief shoots for constant moon presence in 10 years',
 'U.S. Life Expectancy Is Falling – And The 2 Biggest Reasons Why Will Absolutely Stun You…',
 'Ancient Cave Paintings Show Early Humans Understood Complex Astronomy',
 'USA’s ‘First Biometric Terminal’ Ready to Go at Atlanta Airport',
 'The Coldest And Snowiest November Ever? Here Is Why America’s Freakish Weather Is Only Going To Get Worse…',
 'India Invites Other Countries for Venus Mission',
 'Chinese Scientist Behind ‘Gene-Edited Babies’ Claims to Pause Trial After Outcry',
 'Design, Not Accident',
 'Like Humans, Dogs Know When They Don’t Know Enough To Make A Decision',
 'The Truth About China’s Gene Edited Babies',
 'Science Shocker: Adam And 

In [17]:
news_title_list = []
for i in range(len(category_url_list)):
    title_list = get_title(category_url_list[i])
    news_title_list = news_title_list + title_list
    news_title_list = list(set(news_title_list))

In [18]:
len(news_title_list)

319

In [19]:
new_content_list = []
multi_page_url_list = get_url(news_title_list)
for i in range(len(news_title_list)):
    new_content_list.append(get_content(multi_page_url_list[i]))

In [20]:
#Dataframe: News | URL | Content
data = {'Title':news_title_list,'URL':multi_page_url_list,'Content':new_content_list}
df = pd.DataFrame(data)
df = df[['Title', 'URL', 'Content']]
df.head()

,Title,URL,Content
0,Massive Dow Drop Incoming – CFO Poll,https://www.infowars.com/Massive-Dow-Drop-Inco...,The Dow plunge isn’t anywhere near done.At lea...
1,Washington Approves Sale of 10 Apache Attack H...,https://www.infowars.com/Washington-Approves-S...,The US government has given the go-ahead to se...
2,Netanyahu to Take Over Defense Job,https://www.infowars.com/Netanyahu-to-Take-Ove...,Israeli Prime Minister Benjamin Netanyahu will...
3,A White House Press Pass Has Nothing to do wit...,https://www.infowars.com/A-White-House-Press-P...,A federal judge today ruled the White House mu...
4,A Chemical Weapon Attack Just Happened In Alep...,https://www.infowars.com/A-Chemical-Weapon-Att...,Owen Shroyer talks with guest Syrian Girl abou...


In [21]:
df.to_csv('result.csv')

## 3. Crawl Breaking News 

In [22]:
url_breaking = 'https://www.infowars.com/breaking-news/'

In [23]:
def get_article_title(url): 
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    us_news_div = soup.find_all('div', re.compile('articles-list'))

    title_list = []
    for i in range(len(us_news_div)):
        us_news_a = us_news_div[i].find_all('a')
        us_news_h3 = us_news_a[0].find_all('h3')

        for index, item in enumerate(us_news_h3[:]):
            title = item.text.strip()
            title_list.append(title)

    return title_list


In [24]:
get_article_title(url_breaking)

['Socialist Losers Given The Wheel Of The Democratic Party']

In [25]:
content_list = []
title_list = get_article_title(url_breaking)
url_list = get_url(title_list)
for i in range(len(title_list)):
    content_list.append(get_content(url_list[i]))

In [26]:
data = {'Title':title_list,'URL':url_list,'Content':content_list}
df = pd.DataFrame(data)
df = df[['Title', 'URL', 'Content']]
df.head()

,Title,URL,Content
0,Socialist Losers Given The Wheel Of The Democr...,https://www.infowars.com/Socialist-Losers-Give...,It only took 24 hours for the smug Democrats a...
